In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import gc
from pathlib import Path

import torch

from course_copilot import transcription, topic_segmentation, summarization

## Configuration

In [3]:
youtube_id = "Jsz4E2iNXUA"
device = "cuda"

audio_files_fpath = Path("../../transcription/audio_files")
transcription_fpath = Path("../../transcription/transcriptions")
whisper_models_fpath = Path("../../transcription/models")

whisper_model = "base"
topic_segmentation_learner_fpath = "../../models/topic_segmentation_deberta_v3_small.pkl"
headline_summarization_learner_fpath = "../../models/{exported_learner.pkl}"
content_summarization_learner_fpath = "../../models/{exported_learner.pkl}"

# smaller = more fine grained topics (more topics predicted) while larger values = less fine grained (less topics predicted)
topic_segmentation_get_preds_kwargs = {"threshold_std_coeff": 1.9}

## Transcription

In [4]:
audio_fpath = transcription.fetch_youtube_audio(youtube_id, audio_files_fpath=audio_files_fpath)

transcription_fpath = transcription.fetch_transcription(
    audio_fpath=audio_fpath,
    transcription_fpath=transcription_fpath,
    model_fpath=whisper_models_fpath,
    model_checkptoint=whisper_model,
    device=device,
)

transcription_df = transcription.transcription_to_df(transcription_fpath)
transcription_df.head()

,elapsed_seconds,timestamp,transcript
0,0.00,00:00:00.000,Let me make sure everything is as it should be.
1,3.56,00:00:03.560,I'm always fascinated by the fact that people are waiting.
2,6.72,00:00:06.720,It's like it's it's so surprising that people are like on here
3,12.70,00:00:12.700,sometimes early for you.
4,14.12,00:00:14.120,It makes sense for my session.


## Topic Segmentation

In [5]:
topic_segmentation_trainer = topic_segmentation.TopicSegmentationModelTrainer(
    experiment_name="deberta_v3_small", train_config=topic_segmentation.TopicSegmentationConfig
)
# results_df, train_df, val_idxs = trainer.train()
inf_learn = topic_segmentation_trainer.load_learner_or_model(
    model_learner_fpath=topic_segmentation_learner_fpath, device=device
)

topic_seg_preds_df, pred_topic_idxs = topic_segmentation_trainer.get_preds(
    inf_learn, transcription_df.copy(), **topic_segmentation_get_preds_kwargs
)

# cleanup resources
del inf_learn
torch.cuda.empty_cache()
gc.collect()

# shows final results
print("# predicted topics: ", len(pred_topic_idxs))
print(pred_topic_idxs[:10])
topic_seg_preds_df.head()

# predicted topics:  41
[0, 44, 113, 129, 133, 145, 150, 155, 162, 170]


,elapsed_seconds,timestamp,transcript,depth_score,threshold,pred_start
0,0.00,00:00:00.000,Let me make sure everything is as it should be.,0.006972,0.022118,True
1,3.56,00:00:03.560,I'm always fascinated by the fact that people are waiting.,0.004309,0.022118,False
2,6.72,00:00:06.720,It's like it's it's so surprising that people are like on here,0.000000,0.022118,False
3,12.70,00:00:12.700,sometimes early for you.,0.003844,0.022118,False
4,14.12,00:00:14.120,It makes sense for my session.,0.005018,0.022118,False


## Summarization

In [6]:
summarization_inf_df = topic_seg_preds_df.copy()

summarization_inf_df["topic_num"] = -1
n_topics = len(pred_topic_idxs)

for i, seg_idx in enumerate(pred_topic_idxs):
    end_idx = pred_topic_idxs[i + 1] if i + 1 <= (n_topics - 1) else None
    summarization_inf_df.loc[seg_idx:end_idx, "topic_num"] = i

summarization_inf_df = summarization_inf_df.groupby(by="topic_num").agg(list).reset_index()
summarization_inf_df["transcript"] = summarization_inf_df["transcript"].apply(
    lambda v: " ".join([str(seq) for seq in v])
)
summarization_inf_df.timestamp = summarization_inf_df.timestamp.apply(lambda v: v[0])
summarization_inf_df.drop(columns=["depth_score", "threshold", "pred_start"], inplace=True)

summarization_inf_df.head()

,topic_num,elapsed_seconds,timestamp,transcript
0,0,"[0.0, 3.56, 6.72, 12.7, 14.12, 15.24, 22.72, 23.2, 25.08, 25.68, 26.96, 27.8, 28.8, 30.28, 32.04, 36.6, 39.12, 42.4, 44.96, 47.44, 51.92, 54.68, 60.64, 63.28, 66.76, 70.56, 71.24, 72.8, 73.56, 74.0, 75.16, 78.2, 80.16, 85.56, 94.28, 95.28, 106.16, 128.04, 129.24, 133.0, 140.68, 141.68, 150.16, 152.96]",00:00:00.000,"Let me make sure everything is as it should be. I'm always fascinated by the fact that people are waiting. It's like it's it's so surprising that people are like on here sometimes early for you. It makes sense for my session. I'm like why are people joining? Awesome. I believe we live. Welcome back. Everyone wait is back. He wasn't well. Last week. Now he's in awesome health. So that's I was happy to hear that. And I hope you are as well because he'll be teaching us how to translate stuff and how to summarize models. If I can learn this from him, I'll take his model and deploy it on my pod..."
1,1,"[154.04, 155.64, 162.56, 163.88, 169.4, 175.52, 176.6, 181.52, 184.76, 189.92, 193.84, 196.68, 200.56, 206.68, 208.56, 211.76, 214.28, 220.44, 221.88, 224.92000000000002, 225.88, 229.52, 231.8, 235.28, 239.07999999999998, 242.68, 246.0, 249.12, 253.4, 256.0, 257.76, 259.6, 264.68, 270.16, 273.88, 280.24, 285.04, 287.12, 292.72, 293.0, 294.96, 295.8, 301.2, 312.6, 314.64, 316.04, 317.96, 320.36, 324.36, 330.36, 333.36, 336.84000000000003, 337.76, 341.76, 345.88, 350.64, 352.72, 355.8, 358.52, 359.44, 364.4, 365.92, 368.4, 369.64, 373.4, 377.24, 379.8, 383.88, 387.16]",00:02:34.040,"In the meantime, I can talk about tricks. I've been trying to learn different tricks shared across NLP, computer vision world. I found out the augmentee library by the Spacey group. Let's see if I can find that up. Here it is. So let's see link to it. I found that to be quite interesting. And a few things I've been trying to learn in the meantime is NLP augmentations. As you can see, I learned a cool trick called back translation. I'm sure everyone knows of it. But when you're trying to perform augmentation, you can translate your original text to foreign language, translate it back. And j..."
2,2,"[390.36, 393.6, 399.64, 402.0, 406.2, 408.36, 411.08, 413.32, 415.64, 420.64, 422.48, 425.64, 430.36, 433.92, 437.76, 440.48]",00:06:30.360,"but we're now looking at some of the major NLP tasks that you can build with using transformers. And then in particular, we're looking at how we can use FASTI to develop these models, train them, deploy them for inference. And their models that are supported in a library built called Blur. So we'll be looking today kind of really diving into the summarization and translation bits. And I combined them because, as you'll see, they're actually very similar. There's little intricacies with building these type of models. And there's minor differences between building a summarization versus a tr..."
3,3,"[442.04, 444.52, 447.84, 454.15999999999997]",00:07:22.040,"So we're going to kind of go through our seven steps that we've used for looking at question answering task and also looking at the token classification task. And along the way, we'll talk about,"
4,4,"[455.84000000000003, 459.96, 462.88, 467.36, 471.72, 474.68, 476.48, 479.04, 480.8, 483.88, 486.96, 489.12]",00:07:35.840,"get a little bit deeper into model selection and also metrics selection. So as you know, when you're actually building a model, the loss is kind of the models way, the models metric, that it understands in terms of improving the quality of your weights. But from a human perspective, what we really care about is metrics. And so regardless of what you're doing, and in particular, when we look at summarization and translation, the metrics that we're going to want to use are going to be different for these tasks."


In [7]:
# TODO: Kurian to add his magic here after he adds his models to /models ...